# Recursive Character Text Splitter
`RecursiveCharacterTextSplitter`는  **가장 지능적인** 텍스트 분할 도구입니다.  
CharacterTextSplitter의 단일 separator 방식과 달리, **여러 separator를 순차적으로 시도**하여 가장 자연스러운 분할점을 찾습니다.

### 주요 특징
1. **지능적 분할**: 여러 separator를 우선순위에 따라 순차 적용합니다
2. **자연스러운 경계**: 문단 → 문장 → 단어 순으로 자연스러운 분할을 수행합니다
3. **유연한 설정**: separator 목록을 사용자가 정의 가능합니다
4. **실무 최적화**: 대부분의 RAG 시스템에서 권장되는 방식입니다


In [ ]:
comparison_text = """제1장: 머신러닝 기초

머신러닝은 인공지능의 핵심 분야로, 컴퓨터가 데이터로부터 자동으로 학습하여 패턴을 찾고 예측을 수행하는 기술입니다.

주요 학습 방법들:
지도학습 - 레이블된 데이터를 사용하여 모델을 훈련시키는 방법입니다. 분류와 회귀 문제에 사용됩니다. 예시로는 이메일 스팸 필터링, 주식 가격 예측, 고객 세분화, 의료 진단 보조 시스템 등이 있습니다.
비지도학습 - 레이블이 없는 데이터에서 숨겨진 패턴을 찾는 방법으로 클러스터링과 차원 축소에 활용됩니다.

강화학습은 에이전트가 환경과 상호작용하면서 보상을 통해 최적의 정책을 학습하는 방법입니다. 게임 AI, 로봇 제어, 자율주행 등에 활용됩니다.

제2장: 딥러닝 아키텍처

딥러닝은 인공 신경망을 기반으로 한 머신러닝의 하위 분야입니다. CNN은 합성곱 신경망으로 이미지 인식에 특화되어 있습니다. RNN은 순환 신경망으로 시계열 데이터와 자연어 처리에 적합합니다. Transformer는 어텐션 메커니즘을 기반으로 한 최신 아키텍처로 GPT, BERT, T5 등의 기반이 됩니다.
"""


## 2. CharacterTextSplitter와의 비교

`RecursiveCharacterTextSplitter`의 핵심 차이점을 이해해보겠습니다.

### 주요 차이점

| 구분 | CharacterTextSplitter | RecursiveCharacterTextSplitter |
|------|----------------------|-------------------------------|
| **Separator** | 단일 separator | 여러 separator 목록 (우선순위) |
| **분할 방식** | 고정된 기준으로 분할 | 적응적으로 최적 기준 선택 |
| **자연스러움** | 기계적 분할 | 의미 단위 보존하며 분할 |
| **사용 복잡도** | 간단 | 약간 복잡하지만 더 지능적 |
| **결과 품질** | 기본적 | 더 자연스럽고 의미 있는 분할 |

### Recursive 동작 원리

1. **1단계**: 첫 번째 separator(`"\n\n"`)로 분할 시도
2. **2단계**: 청크가 너무 크면 두 번째 separator(`"\n"`)로 재분할
3. **3단계**: 여전히 크면 세 번째 separator(`". "`)로 재분할
4. **4단계**: 마지막으로 네 번째 separator(`" "`)로 단어 단위 분할
5. **최종**: 모든 separator를 시도해도 크면 강제로 문자 단위 분할


**실습**  

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

character_splitter = CharacterTextSplitter(
    separator="\n\n",       # 문단 구분만 사용
    chunk_size=100,         # 200자 제한
    chunk_overlap=30
)

character_chunks = character_splitter.split_text(comparison_text)

print(f"총 {len(character_chunks)}개 청크 생성\n")

for i, chunk in enumerate(character_chunks, 1):
    chunk_length = len(chunk)
    print(f"청크 {i} ({chunk_length}자):")
    print(f"'{chunk}'")


단일 문단으로 split을 했을때 청크가 너무 길어지거나 너무 많은 문장이 하나의 청크에 섞이는 문제가 발생한다.  
이럴때 `RecursiveCharacterTextSplitter`를 사용할 수 있다.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

recursive_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " "],  # 문단 → 줄 → 문장 → 단어 순
    chunk_size=100,    
    chunk_overlap=30
)

recursive_chunks = recursive_splitter.split_text(comparison_text)

print(f"\n총 {len(recursive_chunks)}개 청크 생성\n")

for i, chunk in enumerate(recursive_chunks, 1):
    chunk_length = len(chunk)
    print(f"청크 {i} ({chunk_length}자):")
    print(f"'{chunk}'")